# 01 — FMA large setup
End-to-end setup for AudioCraft + the FMA large dataset on RunPod (`/workspace`).


In [1]:
from pathlib import Path
import os

BASE_DIR = Path("/workspace")
DATA_DIR = BASE_DIR / "data" / "fma_large"
RAW_DIR = BASE_DIR / "data" / "fma_raw"
AUDIOCRAFT_REPO_DIR = BASE_DIR / "audiocraft"
EXPERIMENTS_DIR = BASE_DIR / "experiments" / "audiocraft"

SEGMENT_SECONDS = 10
TARGET_SR = 32000
CHANNELS = 1
TRAIN_RATIO = 0.9
RANDOM_SEED = 42
NUM_SAMPLES_TOTAL = None  # keep full archive; set an int to downsample

FMA_ARCHIVE_URLS = [
    os.environ.get("FMA_LARGE_ARCHIVE_URL"),
    "https://os.unil.cloud.switch.ch/fma/fma_large.zip",
    "https://mirror.math.princeton.edu/pub/fma/fma_large.zip",
    "https://huggingface.co/datasets/echonest/fma_large/resolve/main/fma_large.zip",
]
FMA_ARCHIVE_URLS = [u for u in FMA_ARCHIVE_URLS if u]

WAV_DIR = DATA_DIR / "wav_32k_mono"
SEGMENTS_DIR = DATA_DIR / "segments_10s"
MANIFEST_DIR = DATA_DIR / "manifests"
EGS_TRAIN = DATA_DIR / "egs" / "train"
EGS_VALID = DATA_DIR / "egs" / "valid"

for p in (DATA_DIR, RAW_DIR, WAV_DIR, SEGMENTS_DIR, MANIFEST_DIR, EGS_TRAIN, EGS_VALID, EXPERIMENTS_DIR):
    p.mkdir(parents=True, exist_ok=True)

print("BASE_DIR:", BASE_DIR)
print("Using URLs (in order):", FMA_ARCHIVE_URLS)


BASE_DIR: /workspace
Using URLs (in order): ['https://os.unil.cloud.switch.ch/fma/fma_large.zip', 'https://mirror.math.princeton.edu/pub/fma/fma_large.zip', 'https://huggingface.co/datasets/echonest/fma_large/resolve/main/fma_large.zip']


## 1) System dependencies
Installs ffmpeg + related libs and shows versions.


In [2]:
# System deps: ffmpeg + unzip + sha1sum
!apt-get update -y
!apt-get install -y ffmpeg unzip coreutils

Get:1 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease [18.1 kB]
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [128 kB]      
Get:3 http://archive.ubuntu.com/ubuntu focal InRelease [265 kB]                
Get:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease [128 kB]        
Get:5 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [4801 kB]
Get:6 http://archive.ubuntu.com/ubuntu focal-backports InRelease [128 kB]
Get:7 http://archive.ubuntu.com/ubuntu focal/universe amd64 Packages [11.3 MB]
Get:8 http://archive.ubuntu.com/ubuntu focal/multiverse amd64 Packages [177 kB]
Get:9 http://archive.ubuntu.com/ubuntu focal/main amd64 Packages [1275 kB]
Get:10 http://archive.ubuntu.com/ubuntu focal/restricted amd64 Packages [33.4 kB]
Get:11 http://archive.ubuntu.com/ubuntu focal-updates/restricted amd64 Packages [4998 kB]
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [4920 kB]
Get:13 http://archive.ubun

## 2) Python dependencies
Upgrades pip/build tools, installs Dora, PyTorch CUDA (cu121), Transformers.


In [ ]:
%%bash
set -euo pipefail
python3 -V

if ! python3 -m pip --version >/dev/null 2>&1; then
  echo "pip not found for python3; installing python3-pip..."
  sudo apt-get update
  sudo apt-get install -y python3-pip
else
  echo "pip already installed:"
  python3 -m pip --version
fi

python -m pip install -U pip setuptools wheel
python -m pip install -U dora-search
python -m pip install -U torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
python -m pip install -U transformers


In [ ]:
import torch
print("torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())


## 3) Clone AudioCraft repo
Fresh clone into `/workspace/audiocraft`.


In [ ]:
# System deps: ffmpeg libraries

!apt-get update -y
!apt-get install -y pkg-config libavformat-dev libavdevice-dev libavfilter-dev libavcodec-dev libavutil-dev libswscale-dev libswresample-dev

In [ ]:
%%bash
set -euo pipefail
AUDIOCRAFT_REPO_DIR="/workspace/audiocraft"
if [ -d "$AUDIOCRAFT_REPO_DIR" ]; then
  rm -rf "$AUDIOCRAFT_REPO_DIR"
fi
git clone https://github.com/facebookresearch/audiocraft.git "$AUDIOCRAFT_REPO_DIR"
ls -la "$AUDIOCRAFT_REPO_DIR/config" | head


## 4) Install AudioCraft
Installs requirements and editable package.


In [ ]:
%%bash
set -euo pipefail
cd /workspace/audiocraft
python -m pip install -r requirements.txt
python -m pip install -e .


In [3]:
%%bash
set -e
apt-get install -y aria2

Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  libaria2-0 libc-ares2 libssh2-1
The following NEW packages will be installed:
  aria2 libaria2-0 libc-ares2 libssh2-1
0 upgraded, 4 newly installed, 0 to remove and 1 not upgraded.
Need to get 1551 kB of archives.
After this operation, 6235 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libc-ares2 amd64 1.15.0-1ubuntu0.5 [36.9 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 libssh2-1 amd64 1.8.0-2.1ubuntu0.1 [75.5 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal/universe amd64 libaria2-0 amd64 1.35.0-1build1 [1082 kB]
Get:4 http://archive.ubuntu.com/ubuntu focal/universe amd64 aria2 amd64 1.35.0-1build1 [356 kB]


debconf: delaying package configuration, since apt-utils is not installed


Fetched 1551 kB in 1s (1204 kB/s)
Selecting previously unselected package libc-ares2:amd64.
(Reading database ... 30797 files and directories currently installed.)
Preparing to unpack .../libc-ares2_1.15.0-1ubuntu0.5_amd64.deb ...
Unpacking libc-ares2:amd64 (1.15.0-1ubuntu0.5) ...
Selecting previously unselected package libssh2-1:amd64.
Preparing to unpack .../libssh2-1_1.8.0-2.1ubuntu0.1_amd64.deb ...
Unpacking libssh2-1:amd64 (1.8.0-2.1ubuntu0.1) ...
Selecting previously unselected package libaria2-0:amd64.
Preparing to unpack .../libaria2-0_1.35.0-1build1_amd64.deb ...
Unpacking libaria2-0:amd64 (1.35.0-1build1) ...
Selecting previously unselected package aria2.
Preparing to unpack .../aria2_1.35.0-1build1_amd64.deb ...
Unpacking aria2 (1.35.0-1build1) ...
Setting up libc-ares2:amd64 (1.15.0-1ubuntu0.5) ...
Setting up libssh2-1:amd64 (1.8.0-2.1ubuntu0.1) ...
Setting up libaria2-0:amd64 (1.35.0-1build1) ...
Setting up aria2 (1.35.0-1build1) ...
Processing triggers for libc-bin (2.31-

In [ ]:
%pip install -U "torch==2.5.1" "torchvision==0.20.1" "torchaudio==2.5.1" --index-url https://download.pytorch.org/whl/cu121
%pip install -U "transformers==4.46.3"
%pip install --force-reinstall "pydantic<2" "pydantic-core<2"

# IMPORTANT: Restart the kernel after running this cell to ensure proper loading of the updated packages

In [ ]:
import sys, subprocess
print("Kernel python:", sys.executable)
subprocess.run([sys.executable, "-m", "pip", "install", "-e", "/workspace/audiocraft"], check=True)
subprocess.run([sys.executable, "-m", "pip", "show", "audiocraft"], check=True)

In [ ]:
import sys
sys.path.insert(0, "/workspace/audiocraft")
import audiocraft
print("audiocraft import: OK", audiocraft.__file__)

## Data download & prep (moved)
Continue with `01b_fma_large_downloader.ipynb` for steps 6–13 (download, segmentation, manifests, config).
